# データ前処理

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
# カラム
correct_columns = [
    "年",
    "月",
    "日",
    "回次",
    "場所",
    "日次",
    "レース番号",
    "レース名",
    "クラスコード",
    "芝・ダ",
    "トラックコード",
    "距離",
    "馬場状態",
    "馬名",
    "性別",
    "年齢",
    "騎手名",
    "斤量",
    "頭数",
    "馬番",
    "確定着順",
    "入線着順",
    "異常コード",
    "着差タイム",
    "人気順",
    "走破タイム",
    "走破時計",
    "補正タイム",
    "通過順1",
    "通過順2",
    "通過順3",
    "通過順4",
    "上がり3Fタイム",
    "馬体重",
    "調教師",
    "所属地",
    "賞金",
    "血統登録番号",
    "騎手コード",
    "調教師コード",
    "レースID",
    "馬主名",
    "生産者名",
    "父馬名",
    "母馬名",
    "母の父馬名",
    "毛色",
    "生年月日",
    "単勝オッズ",
    "馬印",
    "レース印",
    "PCI",
]

In [3]:
# CSVファイルのパスを取得
# {YYYY}/{yy}{開催}{開催場所一文字目}.CSV
# という形式のファイルがある。
# ex1, 2021年東京開催第1回目
#     ->2021/211東.CSV
# ex2, 2025年阪神開催第4回目
#     ->2025/254阪.CSV

# *.csv ファイルを全て取得
csv_dir = Path("../csv/")
csv_files = csv_dir.glob("**/*.CSV")

# 空のDataFrameを作成
df = pd.DataFrame()

# CSVファイルごとに処理
for file in csv_files:
    # 最適化: chunksizeを指定してメモリ効率良く処理
    chunk_iter = pd.read_csv(
        file, chunksize=10000, encoding="cp932", header=None, names=correct_columns
    )
    for chunk in chunk_iter:
        df = pd.concat([df, chunk], ignore_index=True)

In [4]:
# check
df.sample(3)

,年,月,日,回次,場所,日次,レース番号,レース名,クラスコード,芝・ダ,...,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,単勝オッズ,馬印,レース印,PCI
294541,19,12,21,5,阪神,7,5,新馬,15,芝,...,チャンピオンズファ,キズナ,ピースヴィクトリア,マヤノトップガン,芦毛,170507,NaN,,,56.6
165983,15,11,14,3,福島,5,2,500万下,23,ダ,...,大狩部牧場,パイロ,イシノスティール,ラムタラ,栗毛,120522,NaN,,,49.7
187705,15,1,12,1,中山,5,3,未勝利,7,ダ,...,北勝ファーム,マツリダゴッホ,ロンドリーナ,キングヘイロー,鹿毛,120509,NaN,,,46.7


In [5]:
# https://targetfaq.jra-van.jp/faq/detail?site=SVKNEGBV&id=706
# =======================================
#              クラスコード
# =======================================
# 7　　　　  未勝利
# 11	　　未出走
# 15	　　新馬
# 19	　　400万下
# 23	　　500万下
# 39	　　900万下
# 43	　　1000万下
# 63	　　1500万下
# 67	　　1600万下
# 131	　　重賞以外のオープン
# 147	　　グレード無し重賞
# 163	　　G3
# 179	　　G2
# 195	　　G1
# =======================================

df["クラスコード"].unique()

array([  7,  15,  23,  43,  67, 163, 131, 115, 195, 179, 147])

In [6]:
df[['補正タイム', '走破タイム', '走破時計', '上がり3Fタイム']].sample(10)

,補正タイム,走破タイム,走破時計,上がり3Fタイム
103976,0,82.6,1226,33.7
145885,99,72.6,1126,36.8
283916,86,71.3,1113,35.5
379492,43,102.9,1429,41.8
321060,69,118.2,1582,39.2
499159,75,73.8,1138,38.9
98412,0,68.0,1080,36.6
37115,77,108.8,1488,38.8
54067,0,72.8,1128,36.2
169810,84,72.1,1121,37.3


In [7]:
# 扱い易いようにデータの前処理
# 年、月、日からdateカラム（YYMMDD）を生成
df["レース日"] = (
    df["年"].astype(str)
    + df["月"].astype(str).str.zfill(2)
    + df["日"].astype(str).str.zfill(2)
)
# dateカラムをdatetime型に変換
df["レース日"] = pd.to_datetime(df["レース日"], format="%y%m%d", yearfirst=True)

# 生年月日カラムをdatetime型に変換
df["生年月日"] = pd.to_datetime(df["生年月日"], format="%y%m%d", yearfirst=True)

# 不要なカラムを削除
df2 = df.drop(columns=["馬印", "レース印"])  # リストで渡す

In [8]:
df.dtypes

年                    int64
月                    int64
日                    int64
回次                   int64
場所                  object
日次                   int64
レース番号                int64
レース名                object
クラスコード               int64
芝・ダ                 object
トラックコード              int64
距離                   int64
馬場状態                object
馬名                  object
性別                  object
年齢                   int64
騎手名                 object
斤量                 float64
頭数                   int64
馬番                   int64
確定着順                 int64
入線着順                 int64
異常コード                int64
着差タイム              float64
人気順                  int64
走破タイム              float64
走破時計                object
補正タイム                int64
通過順1                 int64
通過順2                 int64
通過順3                 int64
通過順4                 int64
上がり3Fタイム           float64
馬体重                  int64
調教師                 object
所属地                 object
賞金                   int64
血

In [9]:
df.to_csv("../tmp/result.csv")